# Imports

In [7]:
import requests
import time
from multiprocessing import Pool
import json
import re
import pandas as pd

## Script to read from onemap 

In [2]:
def pcode_to_data(pcode):
    if int(pcode) % 1000 == 0:
        print(pcode)
    
    page = 1
    results = []
    
    while True:
        try:
            response = requests.get('http://developers.onemap.sg/commonapi/search?searchVal={0}&returnGeom=Y&getAddrDetails=Y&pageNum={1}'
                                    .format(pcode, page)) \
                               .json()
        except requests.exceptions.ConnectionError as e:
            print('Fetching {} failed. Retrying in 2 sec'.format(pcode))
            
            time.sleep(2)
            continue
            
        results = results + response['results']
        if response['totalNumPages'] > page:
            page = page + 1
        else:
            break
            
    return results

## Define Postal Code Range to Scrape

We search from 0000000 to 999999 but change the start if the process fails/you want to do it in batches

Note it overwrites the existing JSON so you have to save the results first

In [16]:
# pool = Pool(processes=5)

# update starting range if you stopped halfway or to zero if you want to rescan everything

postal_codes = range(957100, 1000000)
postal_codes = ['{0:06d}'.format(p) for p in postal_codes]

## Next cell will run for awhile...

In [6]:
first = True

with open('buildings.json', 'w') as f:
    for p in postal_codes:
        buildings = pcode_to_data(p)
        buildings.sort(key=lambda x: x['SEARCHVAL'])

        for building in buildings:
            if first:
                f.write('[\n')
                first = False
            else:
                f.write(',\n')
            f.write(re.sub('^', '  ', json.dumps(building, indent=2, sort_keys=True), flags=re.MULTILINE))

    f.write(']')

958000
959000
960000
961000
962000
963000
964000
965000
966000
967000
968000
969000
970000
971000
972000
973000
974000
975000
976000
977000
978000
979000
980000
981000
982000
983000
984000
985000
986000
987000
988000
989000
990000
991000
992000
993000
994000
995000
996000
997000
998000
999000


In [14]:
postal = 'buildings.json'

df_postal = pd.read_json(postal)
df_postal

,ADDRESS,BLK_NO,BUILDING,LATITUDE,LONGITUDE,LONGTITUDE,POSTAL,ROAD_NAME,SEARCHVAL,X,Y
0,101A BAYFRONT AVENUE TEMPORARY SITE OFFICE SIN...,101A,TEMPORARY SITE OFFICE,1.275697,103.855652,103.855652,18895,BAYFRONT AVENUE,TEMPORARY SITE OFFICE,30485.511362,28685.612665
1,1 STRAITS BOULEVARD SINGAPORE CHINESE CULTURAL...,1,SINGAPORE CHINESE CULTURAL CENTRE,1.275829,103.849576,103.849576,18906,STRAITS BOULEVARD,SINGAPORE CHINESE CULTURAL CENTRE,29809.365407,28700.236127
2,11A STRAITS BOULEVARD TEMPORARY SITE OFFICE SI...,11A,TEMPORARY SITE OFFICE,1.274950,103.851665,103.851665,18907,STRAITS BOULEVARD,TEMPORARY SITE OFFICE,30041.838898,28602.987244
3,2 CENTRAL BOULEVARD IOI CENTRAL BOULEVARD TOWE...,2,IOI CENTRAL BOULEVARD TOWERS,1.279777,103.851513,103.851513,18916,CENTRAL BOULEVARD,IOI CENTRAL BOULEVARD TOWERS,30024.919852,29136.807026
4,21 PARK STREET DBS MARINA BAY MRT STATION SING...,21,DBS MARINA BAY MRT STATION,1.276427,103.854598,103.854598,18925,PARK STREET,DBS MARINA BAY MRT STATION,30368.205612,28766.381902
...,...,...,...,...,...,...,...,...,...,...,...
142881,100A KRANJI LOOP SINGAPORE 887327,100A,NIL,1.433629,103.758648,103.758648,887327,KRANJI LOOP,100A KRANJI LOOP SINGAPORE 887327,19690.280996,46149.118386
142882,A PASIR PANJANG ROAD SINGAPORE 887328,A,NIL,1.277170,103.795840,103.795840,887328,PASIR PANJANG ROAD,A PASIR PANJANG ROAD SINGAPORE 887328,23828.902741,28848.553424
142883,PULAU BUKOM SINGAPORE 903808,,NIL,1.235596,103.768645,103.768645,903808,PULAU BUKOM,PULAU BUKOM SINGAPORE 903808,20802.263821,24251.557450
142884,GATE C7 AIRPORT CARGO ROAD CHANGI ANIMAL AND P...,GATE C7,CHANGI ANIMAL AND PLANT QUARANTINE CENTRE,1.375285,103.996737,103.996737,918104,AIRPORT CARGO ROAD,CHANGI ANIMAL AND PLANT QUARANTINE CENTRE,46186.488419,39698.194002


In [15]:
df_postal.to_pickle('postal.pkl')